<a href="https://colab.research.google.com/github/tacoculbreth/bioinformatics1/blob/main/bioinfo1_final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#생물정보학 및 실습 1 Final Project (Spring 2025)

This notebook has been created with Google Colab.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!ls -al /content/drive/MyDrive/binfo1-datapack1/

total 7162852
-r-------- 1 root root 1370036258 Apr 27  2023 CLIP-35L33G.bam
-r-------- 1 root root    3118336 Apr 27  2023 CLIP-35L33G.bam.bai
-r-------- 1 root root       7113 May 11  2023 CLIP-let7g.bam
-r-------- 1 root root      14561 May 11  2023 CLIP-let7g-gene.pileup
-r-------- 1 root root    2685065 May 11  2023 CLIP-let7g.pileup
-r-------- 1 root root  883334756 Apr 27  2023 gencode.gtf
-r-------- 1 root root   24065406 Apr 27  2023 read-counts.txt
-r-------- 1 root root        751 Apr 27  2023 read-counts.txt.summary
-r-------- 1 root root 1003658801 Apr 27  2023 RNA-control.bam
-r-------- 1 root root    2276104 Apr 27  2023 RNA-control.bam.bai
-r-------- 1 root root 1260991122 Apr 27  2023 RNA-siLin28a.bam
-r-------- 1 root root    2710744 Apr 27  2023 RNA-siLin28a.bam.bai
-r-------- 1 root root  981684502 Apr 27  2023 RNA-siLuc.bam
-r-------- 1 root root    2606104 Apr 27  2023 RNA-siLuc.bam.bai
-r-------- 1 root root  737352902 Apr 27  2023 RPF-siLin28a.bam
-r-------- 1 r

In [ ]:
!wget --no-check-certificate -O /content/drive/MyDrive/binfo1-datapack1/gencode.gtf.gz http://ftp.ebi.ac.uk/pub/databases/gencode/Gencode_mouse/release_M27/gencode.vM27.annotation.gtf.gz

/content/drive/MyDrive/binfo1-datapack1/gencode.gtf.gz: Read-only file system


In [ ]:
%cd /content/drive/MyDrive/binfo1-work

/content/drive/MyDrive/binfo1-work


In [ ]:
!apt-get update
!apt-get install -y samtools

Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Get:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:3 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Get:4 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Hit:5 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [1,741 kB]
Get:7 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:8 https://r2u.stat.illinois.edu/ubuntu jammy/main all Packages [9,003 kB]
Get:9 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease [18.1 kB]
Hit:10 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:11 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:12 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Get:13 http://security.ubuntu.com/ubuntu

In [ ]:
import subprocess

for i in range(1, 20):
    bam_chr_file = f"CLIP-chr{i}.bam"
    cmd = f"samtools view -b -o {bam_chr_file} CLIP-35L33G.bam chr{i}"
    subprocess.run(cmd, shell=True)

    cmd_count = f"samtools view {bam_chr_file} | wc -l"
    count = subprocess.check_output(cmd_count, shell=True)
    print(f"chr{i}: {count.decode().strip()}")

chr1: 2912078
chr2: 2216553
chr3: 1726860
chr4: 2290061
chr5: 1671681
chr6: 1716693
chr7: 1607064
chr8: 1980078
chr9: 1698825
chr10: 1470488
chr11: 2473335
chr12: 1251248
chr13: 1669714
chr14: 1105276
chr15: 1179938
chr16: 1092107
chr17: 5326606
chr18: 885098
chr19: 823068


In [ ]:
for i in range(1, 20):

    input_bam = f"CLIP-chr{i}.bam"
    output_pileup = f"CLIP-chr{i}.pileup"

    # peiform samtools mpileup
    mpileup_command = f"samtools mpileup {input_bam} > {output_pileup}"
    subprocess.run(mpileup_command, shell=True)

    #check the number of lines
    wc_command = f"wc -l {output_pileup}"
    wc_output = subprocess.check_output(wc_command, shell=True)
    line_count = int(wc_output.decode().split()[0])

    print(f"chr{i}: {line_count} lines")

chr1: 77960011 lines
chr2: 71477096 lines
chr3: 55571207 lines
chr4: 57663905 lines
chr5: 56679177 lines
chr6: 52596274 lines
chr7: 51119086 lines
chr8: 46263546 lines
chr9: 47304486 lines
chr10: 47666391 lines
chr11: 54087894 lines
chr12: 42069818 lines
chr13: 44620190 lines
chr14: 40488435 lines
chr15: 38275840 lines
chr16: 33197654 lines
chr17: 36236232 lines
chr18: 29849906 lines
chr19: 23000226 lines


In [ ]:
#define the function to calculate the entropy

import pandas as pd
import numpy as np
import re

from scipy.stats import entropy

def estimate_shannon_entropy(dna_sequence):
    bases = collections.Counter([tmp_base for tmp_base in dna_sequence])
    # define distribution
    dist = [x/sum(bases.values()) for x in bases.values()]


    entropy_value = entropy(dist, base=2)

    return entropy_value

In [ ]:
import pandas as pd


for i in range(1, 20):

    file_name = f'CLIP-chr{i}.pileup'

    # Read the pileup data for the current chromosome
    chr_pileup = pd.read_csv(file_name, sep='\t', names=['chr', 'startpoint', 'reference', 'readcount', 'seq', 'score'], index_col=None)

    # Filter the data based on readcount >50
    chr_pileup_filtered = chr_pileup[chr_pileup['readcount'] > 50]

    toremove = re.compile(r'[<>$*#+-]|\^.|[0-9]')      #base qc

    chr_pileup_filtered['sequence'] = chr_pileup_filtered['seq'].apply(lambda x: toremove.sub('', x))

    chr_pileup_filtered = chr_pileup_filtered[['chr', 'startpoint', 'sequence']]

    chr_pileup_filtered['length_seq'] = chr_pileup_filtered['sequence'].str.len()    #get the length of seq

    chr_pileup_filtered['entropy'] = chr_pileup_filtered['sequence'].map(lambda x : estimate_shannon_entropy(x))   #calculate the entropy

    chr_pileup_filtered['endpoint'] = chr_pileup_filtered['startpoint'] +  chr_pileup_filtered['length_seq']

    chr_bedgraph_pileup_data = chr_pileup_filtered[['chr','startpoint','endpoint','entropy']]

    chr_bedgraph_pileup_data['-12'] = chr_bedgraph_pileup_data['startpoint'] - 12
    chr_bedgraph_pileup_data['+11'] = chr_bedgraph_pileup_data['startpoint'] + 11

    chr_bedgraph_pileup_fromstart = chr_bedgraph_pileup_data[['chr','-12','+11','entropy']]    #get the interval

    filtered_file_name = f'chr{i}_bedgraph_pileup_f.csv'


    # Save the filtered data as a CSV file
    chr_bedgraph_pileup_f.to_csv(filtered_file_name, index=False)

    # current chromosome
    print(f"Filtered data for chromosome {i}:")

In [ ]:
!pwd


/content
